# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [1]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = 's9xV21k899aPetFLLLj7IQjq2hCprXG0IqymUEy1'  # my api key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [2]:
# Define some util functions

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound, file_path = "files/"): 
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = file_path + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

In [3]:
#MY OWN CONTRIBUTION

# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
MUSIC_LOOPS_DIR = FILES_DIR + '/music_loops/'
DATAFRAME_CSV_FILENAME_MUSIC_LOOPS = 'dataframes/music_loops.csv'

# Build our collection of sounds
freesound_queries_music_loops = [
    {
        'query': 'loop drum',
        'filter': 'duration:[5 TO 10]',
        'num_results': 30, 
    },
    {
        'query': 'music-loop',
        'filter': 'duration:[5 TO 10]',
        'num_results': 30, 
    },
]



def build_sound_collection(freesound_queries, sound_files_path, dataframe_filename):
    # Do all queries and concatenate the results in a single list of sounds
    sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

    # Download the sounds and save them to FILES_DIR folder
    for count, sound in enumerate(sounds):
        print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
        retrieve_sound_preview(sound, sound_files_path)

    # Make a Pandas DataFrame with the metadata of our sound collection and save it
    df =  pd.DataFrame([make_pandas_record(s, sound_files_path) for s in sounds])
    df.to_csv(dataframe_filename)
    print('Saved DataFrame with {0} entries! {1}'.format(len(df), dataframe_filename))

    # Show the contents of our DataFrame (the metadata of our source collection)
    return df, sounds



df, sounds = build_sound_collection(freesound_queries_music_loops, MUSIC_LOOPS_DIR, DATAFRAME_CSV_FILENAME_MUSIC_LOOPS)

df.head(5)


Saved DataFrame with 60 entries! dataframes/music_loops.csv


,freesound_id,license,name,path,tags,username
0,232335,http://creativecommons.org/publicdomain/zero/1.0/,Slow Drum Loop,files/music_loops/232335_4194262-hq.ogg,"[Slow, Drum, Beat, drumkit, kit, loop, drums, ...",Leifgreen
1,320732,http://creativecommons.org/publicdomain/zero/1.0/,time break drum only.mp3,files/music_loops/320732_4548252-hq.ogg,"[Loop, drum-pattern, drum, beat, 115-BPM, kick...",ShadyDave
2,277325,http://creativecommons.org/licenses/by/3.0/,time break.wav,files/music_loops/277325_4548252-hq.ogg,"[atmosphere, Drum, 115BPM, Ambiance, music, Lo...",ShadyDave
3,277375,http://creativecommons.org/licenses/by/3.0/,time break no high note.wav,files/music_loops/277375_4548252-hq.ogg,"[Drum, Electronic, Beats, 115-bpm, loop, Drum-...",ShadyDave
4,263869,http://creativecommons.org/publicdomain/zero/1.0/,Amen Break B (160 BPM),files/music_loops/263869_3095604-hq.ogg,"[drum, bass, n, Jungle, drums, Drum, 160, loop]",Kevcio
